# Different test trying to use zipcodes so I can avoid the googlesearch bot

In this approach, I would get all the child urls from inside the radiology search page on the site and a list of predefined zipcodes from POSPUF or PhysComp. Then, I'd sort out duplicates by url (i.e., id) (concatenating zip codes where multiple were used to find a radiologist) and use this as my list of urls to use in the second part of this program. The reason for using zipcodes is I need to get the results of any unit of variation down enough to recieve all data from the search pages in the site. Unfortunately, the site only populates 50 pages, with 20 profiles on each page, regardless of the search criteria. Therefore, I should be able to keep results to less than 50*20 = 1,000 radiologists per zip code.

NOTE:
I CAN use the page number thing here... e.g., for zipcode 30342, 
page one - https://health.usnews.com/doctors/search?distance=25&location=30342&specialty=Radiology
page two - https://health.usnews.com/doctors/search?distance=25&location=30342&specialty=Radiology&page_num=2
...
page 27 - https://health.usnews.com/doctors/search?distance=25&location=30342&specialty=Radiology&page_num=27

That means I'll need:
1. an outer loop for all zipcodes
2. an inner loop for pages 1-50, with a couple caveats
   * page 1 for a zip code will not have the "&page_num= " bit
       - NVM, if I type &page_num=1 this disappears in the URL but still runs as intended
   * need to build in a way to stop the loop once that zipcode is finished (i.e., if the last page is 27, and the loop moves on to 28, then I don't want it to break
       - NVM, it doesn't actually break. However I'd like to build in a way to stop it from making unnecessary server requests
3. output appended to a df, preferably attached to the zipcode used in the search

In [1]:
# Scraping
from bs4 import BeautifulSoup
import urllib.request
import requests

from time import sleep
from random import randint

import pandas as pd

In [129]:
# Progress counter/tracker
from contextlib import contextmanager
from timeit import default_timer
import sys
from datetime import timedelta

# Define a timer to use with the for-loop nest
@contextmanager
def elapsed_timer():
    start=default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start

# Test w/ single zip and single page

In [2]:
zipcode = '30342'
url = 'https://health.usnews.com/doctors/search?specialty=Radiology&location='+zipcode
user_agent = {'User-agent' : 'Mozilla/5.0'}
page = requests.get(url, headers=user_agent)
soup = BeautifulSoup(page.text, "html.parser")

# Find all HTML relating to Education & Experience (includes med school & residency, licenses, etc.)
li_elements = soup.find_all("li", class_= "item-list__ListItemStyled-sc-18yjqdy-1 fRQMSd")

# Take a look at all the elements
for li_element in li_elements:
    url_element_a = li_element.find('a')
    if url_element_a is not None:
        url_element = url_element_a.get('href')
    print(url_element)
    print() # v-space between elements

/doctors/michael-dille-447907

/doctors/neil-amin-862965

/doctors/pamela-donlan-256487

/doctors/pamela-donlan-256487

/doctors/john-gay-137869

/doctors/kim-gray-67337

/doctors/travis-langley-729504

/doctors/travis-langley-729504

/doctors/jason-oppenheimer-734909

/doctors/neel-patel-690987

/doctors/nirav-patel-119277

/doctors/nirav-patel-119277

/doctors/neil-shah-784970

/doctors/prashant-shankar-779331

/doctors/ashishkumar-parikh-996150

/doctors/ashishkumar-parikh-996150

/doctors/john-grattan-smith-271480

/doctors/courtney-stewart-639821

/doctors/stephen-little-270734

/doctors/stephen-little-270734

/doctors/ariane-neish-269318

/doctors/jane-share-269455

/doctors/thai-trinh-269238

/doctors/thai-trinh-269238

/doctors/amanda-bauer-483431

/doctors/richard-barlow-272286



# Test w/ single zip and multiple pages

In [3]:
### All pages ### 

url_list = []
for num in range(1,51):                       
    page_num = '&page_num='+str(num)
    url = 'https://health.usnews.com/doctors/search?specialty=Radiology&location='+zipcode+page_num
    user_agent = {'User-agent' : 'Mozilla/5.0'}
    page = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(page.text, "html.parser")
    
    # Find all profile list elements on the radiologist search page
    li_elements = soup.find_all("li", class_= "item-list__ListItemStyled-sc-18yjqdy-1 fRQMSd")
    
    # Locate child url in list elements and append to URL list
    for li_element in li_elements:
        url_element_a = li_element.find('a')
        if url_element_a is not None:
            url_element = url_element_a.get('href')
     
        url_list.append(url_element)
        
    # Break the loop if the next page is empty (i.e., if current has less than 26 list elements)
    if len(li_elements) < 26:
        break 
        
    # Iteration delay
    sleep(randint(1,5))

In [4]:
url_list

['/doctors/michael-dille-447907',
 '/doctors/neil-amin-862965',
 '/doctors/pamela-donlan-256487',
 '/doctors/pamela-donlan-256487',
 '/doctors/john-gay-137869',
 '/doctors/kim-gray-67337',
 '/doctors/travis-langley-729504',
 '/doctors/travis-langley-729504',
 '/doctors/jason-oppenheimer-734909',
 '/doctors/neel-patel-690987',
 '/doctors/nirav-patel-119277',
 '/doctors/nirav-patel-119277',
 '/doctors/neil-shah-784970',
 '/doctors/prashant-shankar-779331',
 '/doctors/ashishkumar-parikh-996150',
 '/doctors/ashishkumar-parikh-996150',
 '/doctors/john-grattan-smith-271480',
 '/doctors/courtney-stewart-639821',
 '/doctors/stephen-little-270734',
 '/doctors/stephen-little-270734',
 '/doctors/ariane-neish-269318',
 '/doctors/jane-share-269455',
 '/doctors/thai-trinh-269238',
 '/doctors/thai-trinh-269238',
 '/doctors/amanda-bauer-483431',
 '/doctors/richard-barlow-272286',
 '/doctors/lynn-baxter-271854',
 '/doctors/lisa-bennett-879050',
 '/doctors/leeanna-dick-272224',
 '/doctors/leeanna-dick-2

In [5]:
# Not sure where I was going with this. Probably intended to use it when I figured everything out and need to populate a DF
# NB: Don't run this until fully building it out and I need it below.

# url_dict = {}
# for i in range(0,len(url_list)):
#     url_dict[i] = {
#         'child_url': url_list[i],
#         'zip_searched': zipcode
#     }
# url_dict

# Testing with just two zips 

In [47]:
## Need to get zip column in, formated z5.
# Import zip column
initial_data = pd.read_csv(r"C:\Users\ssantavicca3\Documents\Work Files & Folders\RadiologyTrainees_NotABot\initial_data.csv")
initial_data.dropna(inplace=True)
initial_data["zip"] = initial_data["zip"].astype(int)

# Format zip column with leading zeros and sort out dups
zip_str = [str(x) for x in initial_data["zip"][initial_data["Year"] == 2019]]
zip_lst_b = [x.zfill(5) for x in zip_str]
zip_lst = list(set(zip_lst_b))

# Report...
print("There are "+str(len(zip_lst_b))+" total zips in 2019, with "+str(len(zip_lst))+" being unique.")

There are 10374 total zips in 2019, with 1766 being unique.


In [103]:
zip_lst[0:10]

['90067',
 '38163',
 '92122',
 '02180',
 '07093',
 '55905',
 '06901',
 '20037',
 '52501',
 '14569']

In [174]:
### All pages ### 
import sys

url_list = []
with elapsed_timer() as elapsed:
    zip_count = 0
    for zipcode in zip_lst[599:]:              #CHANGE THIS BACK TO JUST "ZIP_LST" AFTER FINISHING ITERATIONS
        for num in range(1,51):                       
            page_num = '&page_num='+str(num)
            url = 'https://health.usnews.com/doctors/search?specialty=Radiology&location='+zipcode+page_num
            user_agent = {'User-agent' : 'Mozilla/5.0'}
            page = requests.get(url, headers=user_agent)
            soup = BeautifulSoup(page.text, "html.parser")

            # Find all profile list elements on the radiologist search page
            li_elements = soup.find_all("li", class_= "item-list__ListItemStyled-sc-18yjqdy-1 fRQMSd")

            # Locate child url in list elements and append to URL list
            for li_element in li_elements:
                url_element_a = li_element.find('a')
                if url_element_a is not None:
                    url_element = url_element_a.get('href')

                # Populate sub dictionary with child url and zipcode used in search
                url_dict = {} 

                url_dict['zipcode'] = zipcode
                url_dict['url'] = url_element

                url_list.append(url_dict)

            # Break the loop if the next webpage is empty (i.e., if current has less than 26 list elements; 20/page+6dups)
            if len(li_elements) < 26:
                break 

            # Iteration delay
            sleep(randint(1,3))
            
        # Counter and timer for progress checks  
        zip_count += 1
        if (zip_count < 100) & (zip_count % 10 == 0): 
            print("Iteration (zipcodes): "+str(zip_count)+" ----- Time Elapsed: "+str(timedelta(seconds=round(elapsed()))))
            sys.stdout.flush() # force the print (as defined) before kernel completes
        elif zip_count % 100 == 0:
            print("Iteration (zipcodes): "+str(zip_count)+" ----- Time Elapsed: "+str(timedelta(seconds=round(elapsed()))))
            sys.stdout.flush()
            
            # Save intermediate output incase of crash or timeout
            filename = "datadump/first stage/saved_list_iter"+str(zip_count/100)+".txt"
            with open(filename, 'w') as f:
                for item in url_list:
                    f.write(f'{item}\n')
                    sys.stdout.flush()

Iteration (zipcodes): 10 ----- Time Elapsed: 0:09:46
Iteration (zipcodes): 20 ----- Time Elapsed: 0:21:31
Iteration (zipcodes): 30 ----- Time Elapsed: 0:29:54
Iteration (zipcodes): 40 ----- Time Elapsed: 0:36:13
Iteration (zipcodes): 50 ----- Time Elapsed: 0:47:30
Iteration (zipcodes): 60 ----- Time Elapsed: 0:53:52
Iteration (zipcodes): 70 ----- Time Elapsed: 1:03:46
Iteration (zipcodes): 80 ----- Time Elapsed: 1:11:18
Iteration (zipcodes): 90 ----- Time Elapsed: 1:22:09
Iteration (zipcodes): 100 ----- Time Elapsed: 1:31:29
Iteration (zipcodes): 200 ----- Time Elapsed: 3:28:41
Iteration (zipcodes): 300 ----- Time Elapsed: 5:07:36
Iteration (zipcodes): 400 ----- Time Elapsed: 6:48:47
Iteration (zipcodes): 500 ----- Time Elapsed: 8:47:44
Iteration (zipcodes): 600 ----- Time Elapsed: 10:30:53
Iteration (zipcodes): 700 ----- Time Elapsed: 12:24:44
Iteration (zipcodes): 800 ----- Time Elapsed: 14:08:04
Iteration (zipcodes): 900 ----- Time Elapsed: 15:25:42
Iteration (zipcodes): 1000 ----- 

In [82]:
url_list[0:3]                

1300

In [120]:
# POST-PROGRAM TESTING: Count unique zips in the new list of dictionaries

import numpy as np

test_list = []
for i in range(0, len(url_list)):
    test_list.append(url_list[i]["zipcode"])
    
def unique(lst):
    x = np.array(lst)
    print(np.unique(x))

unique(test_list)

['02180' '38163' '90067' '92122']


In [172]:
# Saving the current URL-list (3.2.22)
import csv

# .csv header
fieldnames = ['zipcode', 'url']
# .csv data
rows = [i for i in url_list]

with open('datadump/first stage/url_list.csv', 'w', encoding = 'UTF8', newline = '') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

### Logging scraper terminal output

STARTED AT 2:02PM ON TUESDAY 3/1/22

Iteration (zipcodes): 10----- Time Elapsed: 0:13:09
Iteration (zipcodes): 20----- Time Elapsed: 0:25:49
Iteration (zipcodes): 30----- Time Elapsed: 0:34:07
Iteration (zipcodes): 40----- Time Elapsed: 0:45:37
Iteration (zipcodes): 50----- Time Elapsed: 0:48:28
Iteration (zipcodes): 60----- Time Elapsed: 0:56:44
Iteration (zipcodes): 70----- Time Elapsed: 1:07:04
Iteration (zipcodes): 80----- Time Elapsed: 1:18:35
Iteration (zipcodes): 90----- Time Elapsed: 1:28:46
Iteration (zipcodes): 100----- Time Elapsed: 1:39:10
Iteration (zipcodes): 200----- Time Elapsed: 3:16:39
Iteration (zipcodes): 300----- Time Elapsed: 5:06:54
Iteration (zipcodes): 400----- Time Elapsed: 6:59:17
Iteration (zipcodes): 500----- Time Elapsed: 8:48:33

ConnectionError, MaxRetryError (Errno 1101, getaddrinfo failed)

STARTED AT 7:43AM ON WEDNESDAY 3/2/22

Iteration (zipcodes): 10 ----- Time Elapsed: 0:11:08
Iteration (zipcodes): 20 ----- Time Elapsed: 0:20:49
Iteration (zipcodes): 30 ----- Time Elapsed: 0:32:04
Iteration (zipcodes): 40 ----- Time Elapsed: 0:42:41
Iteration (zipcodes): 50 ----- Time Elapsed: 0:59:03
Iteration (zipcodes): 60 ----- Time Elapsed: 1:08:21
Iteration (zipcodes): 70 ----- Time Elapsed: 1:25:11
Iteration (zipcodes): 80 ----- Time Elapsed: 1:33:39
Iteration (zipcodes): 90 ----- Time Elapsed: 1:44:49
Iteration (zipcodes): 100 ----- Time Elapsed: 1:54:17

FileNotFoundError: (Errno 2) No such file or directory: 'datadump/first stage/url_list.csv/saved_list_iter1.0.txt'

STARTED AT 9:45AM ON WEDNESDAY 3/2/22

In [175]:
url_list[-1]
len(url_list)

578901